In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import sklearn.preprocessing as preprocessing
from sklearn.neighbors import KNeighborsClassifier

from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression


#　Load training data
train = pd.read_csv('train.csv', parse_dates = ['Dates'])

### LabelEncoder testing
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
target = le.fit_transform(train["Category"])
# le.fit([4,5,5,6,3,2,1,1])
# le.transform([4,5,5,6,3,2,1,1])

In [3]:
train.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [4]:
days = pd.get_dummies(train.DayOfWeek)
hour = train.Dates.dt.hour
hour = pd.get_dummies(hour)
district = pd.get_dummies(train.PdDistrict)

train = pd.concat([days, hour, district], axis = 1)

In [5]:
### Load test data
test =  pd.read_csv('test.csv', parse_dates = ['Dates'])

days = pd.get_dummies(test.DayOfWeek)
hour = test.Dates.dt.hour
hour = pd.get_dummies(hour)
district = pd.get_dummies(test.PdDistrict)

test = pd.concat([days, hour, district], axis = 1)

In [6]:
### Validation
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.33, random_state=42)

model = BernoulliNB()
model.fit(X_train, y_train)
predicted = np.array(model.predict_proba(X_test))
log_loss(y_test, predicted)

# model = LogisticRegression(C = .01).fit(X_train, y_train)
# predicted = np.array(model.predict_proba(X_test))
# log_loss(y_test, predicted)

In [12]:
model = BernoulliNB().fit(train, target)
predicted = model.predict_proba(test)

In [13]:
### Write to output file
submission = pd.DataFrame(data = predicted, columns = le.classes_)
submission.to_csv("sub.csv", index = True, index_label = "Id")

In [14]:
submission.head()

,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0.006921,0.130651,0.000022,0.000933,0.029050,0.002047,0.006871,0.032810,0.005735,0.000204,...,0.000134,0.003675,0.000175,0.029836,8.335391e-07,0.003378,0.088410,0.119041,0.039156,0.022786
1,0.006921,0.130651,0.000022,0.000933,0.029050,0.002047,0.006871,0.032810,0.005735,0.000204,...,0.000134,0.003675,0.000175,0.029836,8.335391e-07,0.003378,0.088410,0.119041,0.039156,0.022786
2,0.001811,0.090200,0.000031,0.000224,0.038466,0.003740,0.008674,0.027546,0.007490,0.000230,...,0.000044,0.005844,0.000280,0.020433,2.325302e-07,0.003543,0.074745,0.085124,0.035126,0.008132
3,0.003003,0.123099,0.000023,0.000968,0.026839,0.001756,0.008884,0.018424,0.004918,0.000173,...,0.000149,0.003523,0.000359,0.026149,1.321607e-07,0.002377,0.098499,0.170349,0.024274,0.016148
4,0.003003,0.123099,0.000023,0.000968,0.026839,0.001756,0.008884,0.018424,0.004918,0.000173,...,0.000149,0.003523,0.000359,0.026149,1.321607e-07,0.002377,0.098499,0.170349,0.024274,0.016148
